# Maximize depots in harbour area
1. Export the island with the [Savegame Visualizer](https://github.com/NiHoel/Anno1800SavegameVisualizer). Note: Not all harbour areas are correct. If in doubt check the correct empty island layouts and correct differences: https://drive.google.com/drive/folders/1-KVg2FG2W0lHetd5AYNvN_VOYc5Fcb-s?usp=sharing
2. Prepare the layout in Anno Designer: Fill blocked harbour areas with black tiles. Put a single blue tile in each harbour area where depots should be placed. 
   * The single blue tile must have the the same color as the line that separates land and harbour.
   * Ensure that the harbour area has an outline without gaps


![Exported harbour](imgs/harbour_exported.png)   ![Prepared harbour](imgs/harbour_prepared.png)

3. Run the program


![Run all cells](imgs/run_all.png) ![Restart kernel](imgs/restart_kernel.png)


If you start with an empty island and want to optimizer to avoid placing depots on the blue coast line (if possible), replace the following line by: `AVOID_COASTLINE = True`

In [ ]:
AVOID_COASTLINE = False

In [ ]:
import importlib
from IPython.display import clear_output

if importlib.util.find_spec("pulp") is None :
    %pip install pulp
    clear_output(wait=True) 
    
from pulp import *

available_solvers = listSolvers(onlyAvailable=True)
clear_output(wait=True) 

if "GUROBI_CMD" in available_solvers :
    SOLVER = "GUROBI"
elif "CPLEX_CMD" in available_solvers :
    SOLVER = "CPLEX"
else :
    from tools.fscip_api import FSCIP_CMD
    from subprocess import Popen, PIPE, STDOUT
    SOLVER = "FSCIP"

import ipywidgets as widgets
    
import numpy as np
from scipy import ndimage
import pulp
from pulp import *
from collections import deque

import copy
import json
import math
import pathlib
import PIL.Image
import IPython.display
import shutil

time_limit = 10 * 60 # 10 min <-- increase this for better results

width = 10
height = 4
radius = 20
weight_coast = -0.0001

def pos(obj) :
    return np.array([int(c) for c in obj['Position'].split(',')]) 

def size(obj) :
    return np.array([int(c) for c in obj['Size'].split(',')])

#simple image scaling to (nR x nC) size
def scale(im, nR, nC):
    nR0 = len(im)     # source number of self.rows 
    nC0 = len(im[0])  # source number of columns 
    return [[ im[int(nR0 * r / nR)][int(nC0 * c / nC)]  
             for c in range(nC)] for r in range(nR)]

class Layout :
    def __init__(self, ad_json) :
        self.ad_json = ad_json
        self.dst_json = copy.deepcopy(ad_json)
        self.coast = {"A":255,"R":30,"G":144,"B":255} # part of harbour area
        self.harbour = {"A":255,"R":192,"G":192,"B":192} #outline
        self.blocked = {"A":255,"R":0,"G":0,"B":0}
        self.blocked2 = {"A":255,"R":169,"G":169,"B":169}
        self.river = {"A":255,"R":0,"G":206,"B":209}
        self.river_slot = {"A":255,"R":65,"G":105,"B":225}
        self.mine = {"A":255,"R":160,"G":82,"B":45}
        self.mine2 = {"A":255,"R":115,"G":58,"B":32}
        self.default_object = {"Identifier":"BlockTile_1x1","Label":"","Position":"1,1","Size":"1,1","Icon":None,"Template":"Blocker","Color":{"A":255,"R":255,"G":255,"B":255},"Borderless":False,"Road":False,"Radius":0.0,"InfluenceRange":-2.0,"PavedStreet":False,"BlockedAreaLength":0.0,"BlockedAreaWidth":0.0,"Direction":"Up"}
       

        self.tl = np.array([99999999, 999999999])
        self.br = np.array([-99999999, -999999999])
        one = np.array([1,1])
        for obj in ad_json['Objects'] :
            self.tl = np.minimum.reduce([self.tl, pos(obj)])
            self.br = np.maximum.reduce([self.br, pos(obj) + size(obj)])
            
        self.tl = self.tl - one
        self.br = self.br + one

        dim = self.br - self.tl
        self.area = np.zeros((dim[1], dim[0]), dtype=int)
        

        for obj in ad_json['Objects'] :
            p = pos(obj) - self.tl
            c = obj['Color']
            s = size(obj)
            
            if (s[0] == 1 or s[1] == 1) and c == self.river_slot :
                continue
                
         
            if not np.array_equal(s,one) :
                for x in range(p[0], p[0] + s[0]) :
                    for y in range(p[1], p[1] + s[1]) :
                        self.area[y][x] = -1
                continue

            if c == self.blocked or c == self.river or c == self.mine or c == self.mine2 or c == self.blocked2:
                self.area[p[1]][p[0]] = -1
            elif c == self.coast :
                self.area[p[1]][p[0]] = 2
            elif c == self.harbour :
                self.area[p[1]][p[0]] = 1
            else:
                self.area[p[1]][p[0]] = -1

                
           
        self.rows = len(self.area)
        self.cols = len(self.area[0])
        
        self.cluster_count = 0
        self.clusters = np.zeros((dim[1], dim[0]), dtype=int)
        
        for obj in ad_json['Objects'] :
            p = pos(obj) - self.tl
            if not self.area[p[1]][p[0]] == 2 :
                continue
                
            count_non_zero = 0
            for y in range(p[1] - 1, p[1]+2):
                for x in range(p[0] - 1, p[0] + 2):
                    if not self.area[y,x] == 0:
                        count_non_zero += 1
                        
            if count_non_zero == 1:
                self.cluster_count += 1
                self.fill(p, 3, self.area, self.area, lambda x : x == 0)
                self.area[p[1],p[0]] = 3
                self.fill(p, self.cluster_count, self.area, self.clusters, lambda x : x == 2 or x == 3)
                    
        
    def draw(self, fmt='png', filename=None):
        a = np.uint8(np.clip(scale((self.area+10*self.clusters)*50+50,10*self.rows,10*self.cols) , 0, 255))
        image_data = io.BytesIO()
        PIL.Image.fromarray(a).save(image_data, fmt)
        if filename is None:
            IPython.display.display(IPython.display.Image(data=image_data.getvalue()))
        else:
            with open(filename, 'w') as f:
                image_data.seek(0)
                shutil.copyfileobj(image_data, f)
                
    def fill(self, root, cluster_id, source, target, includes) :
        queue = deque([np.array(root)])
        at = lambda container, pos : container[pos[1]][pos[0]]
        
        while len(queue) > 0 :
            center = queue.popleft()
            for direction in [np.array([1,0]),np.array([0,1]),np.array([-1,0]),np.array([0,-1])] :
                neighbor = center + direction 
                if not at(target, neighbor) == 0 or not includes(at(source, neighbor)):
                    continue
                
                target[neighbor[1]][neighbor[0]] = cluster_id
                queue.append(neighbor)
                

class LPDepot : 
    def __init__(self, layout, cluster_id = 0) :
        self.layout = layout
        self.prob = LpProblem("Harbour layout", LpMaximize)
        self.weights_profit = {}
        self.cluster_id = cluster_id
        self.width = 10
        self.height = 4
        
        rows = self.layout.rows
        cols = self.layout.cols
        area = self.layout.area
        weights_profit = self.weights_profit
        prob = self.prob
        
        #run optimization         
        horizontal = self.get_filler(self.width, self.height)
        vertical = self.get_filler(self.height, self.width)

        for x in range(cols):
            for y in range(rows) :
                if horizontal[y][x] is None or vertical[y][x] is None:
                    continue

                name = "Sum_{}_{}".format(x, y)
                prob.constraints[name] = LpConstraint(LpAffineExpression({horizontal[y][x] : 1, vertical[y][x] : 1}), LpConstraintLE, name, 1)

        prob.objective = LpAffineExpression(weights_profit)
     
    def optimize(self):
        prob = self.prob
        
        if SOLVER == "GUROBI":
            self.status = prob.solve(apis.GUROBI_CMD(timeLimit = time_limit, gapAbs = abs(weight_coast)))
        elif SOLVER == "CPLEX":
            self.status = prob.solve(apis.CPLEX_CMD(timeLimit = time_limit, gapAbs = abs(weight_coast)))
        else:
            solver = FSCIP_CMD(msg=False, timeLimit = time_limit, path=os.getcwd() + "/tools/fscip.exe")
            self.status = solver.actualSolve(prob)
            
        self.coast_count = 0
        self.occupied_coast_count = 0
        
        for variable in self.weights_profit :
            #print(variable.name, value(variable))
            depot,w,x,y = variable.name.split("_")
            
            if depot == "Occupied" :
                self.coast_count += 0.5
                
                if value(variable) > 0 :
                    self.occupied_coast_count += 1              

        
    def color_depots(self):
        layout = self.layout
        to_pixel = lambda val : [255*(val-1),255*(val-1),255*(val-1)]

        depots = []
        for y in range(layout.rows) :
            row = []
            for x in range(layout.cols):
                row.append(to_pixel(layout.area[y][x]))

            depots.append(row)

        #print(depots)
        for variable in self.weights_profit :
            if value(variable) == 0:
                continue

            #print(variable.name)
            depot,w,x,y = variable.name.split("_")
            if not depot == "Depot" :
                continue

            w = int(w)
            x = int(x)
            y = int(y)
            h = self.height if self.width == w else self.width

            for x_ in range(x,min(layout.cols,x + w)):
                    for y_ in range(y, min(layout.rows, y + h)) :
                        depots[y_][x_] = [30*(((x+y)/3)%5)+50,30*((x/2)%5)+50,30*((y/2)%5)+50]

        return depots



    def get_filler(self, width, height) :
        rows = self.layout.rows
        cols = self.layout.cols
        area = self.layout.area
        weights_profit = self.weights_profit
        prob = self.prob

        variables = [[None for i in range(cols)] for j in range(rows)] 
        #filler = [][[LpVariable("Occupied_{}_{}_{}".format(width, i,j), cat='Binary') if area[j][i] else None  for i in range(cols)] for j in range(rows)] 

        filler = []
        for y in range(rows) :
            row = []
            for x in range(cols):
                if area[y][x] >= 2 and self.layout.clusters[y][x] == self.cluster_id:
                    var = LpVariable("Occupied_{}_{}_{}".format(width, x,y), cat='Binary')
                    if area[y][x] == 2 and AVOID_COASTLINE:
                        weights_profit[var] = weight_coast
                    row.append(var)
                else :
                    row.append(None)

            filler.append(row)


        #print(filler)

        for x in range(cols):
            for y in range(rows) :
                count = 0

                for x_ in range(x,min(cols,x + width)):
                    for y_ in range(y, min(rows, y + height)) :
                        if area[y_][x_] >= 2 and self.layout.clusters[y][x] == self.cluster_id:
                            count = count + 1

                if count < width * height:
                    continue


                variables[y][x] = LpVariable("Depot_{}_{}_{}".format(width, x,y), cat='Binary')
                weights_profit[variables[y][x]] = 1

        for x in range(cols):
            for y in range(rows) :

                area_constraint = {}


                for x_ in range(x,min(cols,x + width)):
                    for y_ in range(y, min(rows, y + height)) :

                        #print(filler[y_][x_], variables[y_][x_], (filler[y_][x_] == None or variables[y_][x_] == None))                 
                        if not variables[y_][x_] is None:
                            area_constraint[variables[y_][x_]] = 1

                        if not (filler[y_][x_] is None or variables[y][x] is None) and (x_ == x or y_ == y or x_ == x + width-1 or y_ == y + height-1):
                            name = "Depot_{}_{}_{}_occupies_{}_{}".format(width, x,y,x_, y_)
                            #print(name)
                            prob.constraints[name] = LpConstraint(LpAffineExpression({variables[y][x] : -1, filler[y_][x_] : 1}), LpConstraintGE, name, 0)

                length = len(list(area_constraint.keys()))
                #print("overlap_{}_{}_{}".format(width, x, y), length)

                if length > 0 :
                    name = "overlap_{}_{}_{}".format(width, x, y)
                    prob.constraints[name] = LpConstraint(LpAffineExpression(area_constraint), LpConstraintLE, name, 1)


        return filler

    def draw(self,fmt='png', filename=None):
        a = np.uint8(np.clip(scale(self.color_depots(),10*self.layout.rows,10*self.layout.cols) , 0, 255))
        image_data = io.BytesIO()
        PIL.Image.fromarray(a).save(image_data, fmt)
        if filename is None:
            IPython.display.display(IPython.display.Image(data=image_data.getvalue()))
        else:
            with open(filename, 'w') as f:
                image_data.seek(0)
                shutil.copyfileobj(image_data, f)
                
    def add_to_ad(self) :
        tl = self.layout.tl
        
        for variable in self.weights_profit :
            if value(variable) == 0:
                continue
            
            #print(variable.name)
            depot,w,x,y = variable.name.split("_")
            if not depot == "Depot" :
                continue

            w = int(w)
            x = int(x)
            y = int(y)
            h = self.height if self.width == w else self.width

            color = [30*(((x+y)/3)%5)+50,30*((x/2)%5)+50,30*((y/2)%5)+50]
            obj = copy.deepcopy(self.layout.default_object)
            obj["Identifier"] = "Harbor_01 (Depot)"
            obj["Position"] = "{},{}".format(x + tl[0], y + tl[1])
            obj["Size"] = "{},{}".format(w, h)
            obj["Icon"] = "A7_Depot"
            obj["Template"] = "HarborDepot"
            obj["Color"] = {"A": 255, "R": int(color[0]), "G": int(color[1]), "B": int(color[2])}
            obj["Direction"] = "Up" if w == width else "Right"

            self.layout.dst_json['Objects'].append(obj)

island = None
txt_status = widgets.Text(value="", description="Status" + ":", disabled=True)
txt_status.layout.width = "100%"
def set_status(txt: str):
    global txt_status
    
    txt_status.value = txt

def callback(btn):
    global island
    
    files = btn_file_chooser.value
    if len(files) == 0:
        return

    file = None
    if type(files) is dict:
        file = list(files.values())[0]
    else:
        file = files[0]
        
    new_island = pathlib.Path(file["metadata"]["name"]).stem
    if island == new_island:
        return
    island = new_island

    #ad = json.loads(codecs.decode(file["content"], encoding="utf-8"))
    #print(file["content"].decode("utf-8"))
    ad = json.loads(file["content"].decode("utf-8"))
    output_path = pathlib.Path(island + "_max_depots.ad").resolve()
    layout = Layout(ad)
    
    if layout.cluster_count == 0:
        set_status("No marked harbour areas found!")
    
    set_status("Harbour areas: " + str(layout.cluster_count))

    for cluster_id in range(1,layout.cluster_count+1) :
        opt = LPDepot(layout, cluster_id)
        opt.optimize()
        set_status("Harbour: {} | Depots: {}".format(opt.cluster_id, math.ceil(value(opt.prob.objective))))
        opt.add_to_ad()
        
    with output_path.open("w") as f :
        json.dump(layout.dst_json, f)
        
    set_status("Saved: " + str(output_path))
    
btn_file_chooser = widgets.FileUpload(accept='.ad', multiple=False)
btn_file_chooser.observe(callback)

set_status("Ready")

display(btn_file_chooser)
display(txt_status)